In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = np.concatenate((x_train,x_test), axis = 0)
x_train.shape

(70000, 28, 28)

## 2D Visualization using PCA

In [ ]:
x = []
for i in range(x_train.shape[0]):
  x.append(x_train[i].flatten())

In [ ]:
type(x)
np.array(x).shape

(70000, 784)

In [ ]:
x_train = np.array(x)

In [ ]:
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
print(x_train.shape)

(70000, 784)


In [ ]:
sample_data = x_train

covar_matrix = np.matmul(sample_data.T, sample_data)

print("The shape of variance matrix = ", covar_matrix.shape)

The shape of variance matrix =  (784, 784)


In [ ]:
from scipy.linalg import eigh

values, vectors = eigh(covar_matrix, eigvals=(754,783))

# print(values)

# print(vectors)

print("Shape of eigen vectors =", vectors.shape)

vectors = vectors.T

print("Updated shape of eigen vectors =",vectors.shape)

Shape of eigen vectors = (784, 30)
Updated shape of eigen vectors = (30, 784)


<ipython-input-9-8fbaffb9f630>:3: DeprecationWarning: Keyword argument 'eigvals' is deprecated in favour of 'subset_by_index' keyword instead and will be removed in SciPy 1.12.0.
  values, vectors = eigh(covar_matrix, eigvals=(754,783))


In [ ]:
new_coordinates = np.matmul(vectors, sample_data.T)

print("resultant new data points shape", vectors.shape, "x", sample_data.T.shape," = ", new_coordinates.shape)

resultant new data points shape (30, 784) x (784, 70000)  =  (30, 70000)


In [ ]:
new_coordinates = new_coordinates.T
new_coordinates.shape

(70000, 30)

In [ ]:
x_train = new_coordinates[:60000]
x_test = new_coordinates[60000:]

## Using ANN(Artificial Neural Network) by SKLearn

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (5, 2), random_state=1)
clf.fit(x_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [ ]:
from sklearn import metrics

In [ ]:
pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test,pred)
print(accuracy)


0.7033


In [ ]:
print(clf.predict(x_test[3000:3001]))
print(y_test[3000:3001])
print(clf.predict_proba(x_test[3000:3001]))

[8]
[6]
[[7.18232301e-02 9.85794982e-03 2.20768112e-01 7.76986202e-03
  3.41119268e-02 9.61154287e-02 2.30105364e-01 7.63618799e-05
  3.22537591e-01 6.83417375e-03]]


## Using ANN(Artificial Neural Network) by Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam, RMSprop
from keras import backend as k
from keras.utils import plot_model, to_categorical

In [ ]:
num_labels = len(np.unique(y_train))

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
batch_size = 128
hidden_units = 256
dropout = 0

In [ ]:
model = Sequential()
model.add(Dense(hidden_units, input_dim = 30))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                2

In [ ]:
# plot_model(model, to_file="mlp-mnist.png", show_shapes=True)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_split=0.2, validation_data=(x_test,y_test), epochs=20, batch_size = batch_size)

Epoch 1/20
469/469 [==============================] - 3s 4ms/step - loss: 0.2632 - accuracy: 0.9183 - val_loss: 0.1610 - val_accuracy: 0.9507
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1282 - accuracy: 0.9596 - val_loss: 0.1365 - val_accuracy: 0.9572
Epoch 3/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0985 - accuracy: 0.9678 - val_loss: 0.1147 - val_accuracy: 0.9639
Epoch 4/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0807 - accuracy: 0.9734 - val_loss: 0.1187 - val_accuracy: 0.9636
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0705 - accuracy: 0.9764 - val_loss: 0.1149 - val_accuracy: 0.9660
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0629 - accuracy: 0.9797 - val_loss: 0.1099 - val_accuracy: 0.9647
Epoch 7/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0547 - accuracy: 0.9820 - val_loss: 0.1045 - val_accuracy: 0.9707
Epoch 